In [1]:
import data, calculations

import requests, urllib
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pypfopt import CLA, plotting
from pypfopt.expected_returns import returns_from_prices, log_returns_from_prices, mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import display_html

InteractiveShell.ast_node_interactivity = "all"
pd.options.display.max_columns = None
pd.options.mode.chained_assignment = None  # default='warn'

%load_ext autoreload
%autoreload 2

## Wikipedia Scraper
We can scrape most of the sectors off wikipedia. 

In [2]:
def join_smp500_together(urls: list, keep_dataframe: pd.DataFrame):
    smp500_dataframes = []
    
    for index, url in enumerate(urls):
        dataframe = pd.read_html(url)[2] if index != 0 else pd.read_html(url)[0]
        
        # Handle pre and post 2018 symbol column name
        if "Ticker symbol" in dataframe.columns:
            dataframe.rename(columns={"Ticker symbol": "Symbol"}, inplace=True)
            
        dataframe["Symbol"] = dataframe["Symbol"].astype(str) + " US Equity"
        dataframe = dataframe[["Symbol", "GICS Sector"]]
            
        # Append US Equity symbols to each entry
        smp500_dataframes.append(dataframe)
        
    smp500_dataframe = pd.concat(smp500_dataframes).drop_duplicates().set_index("Symbol")[["GICS Sector"]]
    return smp500_dataframe.loc[portfolio.columns.intersection(smp500_dataframe.index)]

In [3]:
portfolio = data.import_range(date_from="2015")

In [4]:
smp500 = join_smp500_together([
    "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies",
    "https://web.archive.org/web/20190802035125/https://en.wikipedia.org/wiki/List_of_S%26P_500_companies",
    "https://web.archive.org/web/20190427082605/https://en.wikipedia.org/wiki/List_of_S%26P_500_companies",
    "https://web.archive.org/web/20181008215157/https://en.wikipedia.org/wiki/List_of_S%26P_500_companies",
    "https://web.archive.org/web/20170820205408/https://en.wikipedia.org/wiki/List_of_S%26P_500_companies",
    "https://web.archive.org/web/20160118103416/https://en.wikipedia.org/wiki/List_of_S%26P_500_companies",
    "https://web.archive.org/web/20150818030513/https://en.wikipedia.org/wiki/List_of_S%26P_500_companies",
    "https://web.archive.org/web/20141124110032/http://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
], portfolio)

In [5]:
smp500

,GICS Sector
Symbol,
A US Equity,Health Care
AA US Equity,Materials
AAL US Equity,Industrials
AAP US Equity,Consumer Discretionary
AAPL US Equity,Information Technology
...,...
XYL US Equity,Industrials
YUM US Equity,Consumer Discretionary
ZBH US Equity,Health Care


## Scrape Remaining Data
Some stocks aren't listed on Wikipedia, so we need to get them elsewhere. 

A [medium](https://towardsdatascience.com/beat-the-stock-market-with-machine-learning-d9432ea5241e) article describes how to retreive stock data from an api, Financial Modelling Prep.

In [6]:
def get_json_data(url):
    # Scrape data (which must be json format) from given url
    response = urllib.request.urlopen(url)
    dat = response.read().decode('utf-8')
    return json.loads(dat)

def find_in_json(obj, key):
    # Scan the json file to find the value of the required key
    arr = []

    def extract(obj, arr, key):
        # Recursively search for values of key in json file.
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    results = extract(obj, arr, key)
    return results

def get_sector(stock):
    a = get_json_data(f"https://financialmodelingprep.com/api/v3/profile/{stock}?apikey=dac00d35299c4b74d8ab7ed3a1d946b0")
    return find_in_json(a, "sector")

Remove suffix "US Equity" from ticker strings to match criteria.

In [7]:
diff = list(set(portfolio.columns) - set(smp500.index))

In [8]:
remaining_tickers = [i.replace('US Equity', '').replace('/', '.').strip() for i in diff]

In [9]:
remaining_tickers[:10]

['GHC', 'JBL', 'CR', 'DDS', 'UN', 'SNV', 'PTC', 'DLX', 'NC', 'BGG']

Find all remaining sectors and create a list

In [10]:
remaining_sectors = [get_sector(ticker)[0] for ticker in remaining_tickers]

In [11]:
remaining_sectors

['Consumer Defensive',
 'Technology',
 'Industrials',
 'Consumer Cyclical',
 'Consumer Defensive',
 'Financial Services',
 'Technology',
 'Communication Services',
 'Energy',
 'Industrials',
 'Financial Services',
 'Consumer Cyclical',
 'Basic Materials',
 'Consumer Cyclical',
 'Industrials',
 'Consumer Cyclical',
 'Industrials',
 'Basic Materials',
 'Financial',
 'Industrials',
 'Basic Materials',
 'Consumer Cyclical',
 'Healthcare',
 'Consumer Cyclical',
 'Consumer Defensive',
 'Industrials',
 'Consumer Cyclical',
 'Industrials',
 'Industrials',
 'Consumer Defensive',
 'Basic Materials',
 'Consumer Cyclical',
 'Industrials',
 'Consumer Cyclical',
 'Communication Services',
 'Consumer Cyclical',
 'Communication Services',
 'Basic Materials',
 'Industrials']

Create a list of remianing stocks which do not have an associated sector, changing '.' to '/' to reflect source data naming conventions.

In [12]:
remaining_tickers = [remaining_tickers[i].replace(".", "/") for i in range(len(remaining_tickers))]

In [13]:
remaining_sectors[:5]

['Consumer Defensive',
 'Technology',
 'Industrials',
 'Consumer Cyclical',
 'Consumer Defensive']

In [14]:
final_sectors = pd.DataFrame({"Symbol": remaining_tickers, "GICS Sector": remaining_sectors})
final_sectors["Symbol"] = final_sectors["Symbol"].astype(str) + " US Equity"
final_sectors.set_index("Symbol", inplace=True)

In [15]:
smp500 = pd.concat([smp500, final_sectors])

In [16]:
smp500 = smp500[~smp500.index.duplicated(keep='first')]

In [17]:
smp500.sort_index(inplace=True)

In [18]:
set(smp500.index) == set(portfolio.columns)

True

## Clean Up

In [19]:
smp500["GICS Sector"].value_counts()

Industrials               74
Financials                60
Information Technology    58
Consumer Discretionary    58
Health Care               57
Energy                    33
Real Estate               30
Consumer Staples          29
Utilities                 26
Materials                 22
Communication Services    21
Consumer Cyclical         10
Basic Materials            5
Consumer Defensive         4
Financial Services         2
Technology                 2
Healthcare                 1
Financial                  1
Name: GICS Sector, dtype: int64

In [20]:
new_sectors = {
    "Financial": "Financials", "Technology": "Information Technology",
    "Telecommunication Services": "Communication Services", "Telecommunications Services": "Communication Services",
    "Health Care": "Healthcare", "Consumer Cyclical": "Consumer Discretionary", "Basic Materials": "Materials",
    "Consumer Defensive": "Consumer Staples", "Financial Services": "Financials"
}

sectors_mapper = lambda name: new_sectors[name] if name in new_sectors.keys() else name

In [21]:
smp500["GICS Sector"] = smp500["GICS Sector"].map(sectors_mapper)
smp500

,GICS Sector
Symbol,
A US Equity,Healthcare
AA US Equity,Materials
AAL US Equity,Industrials
AAP US Equity,Consumer Discretionary
AAPL US Equity,Information Technology
...,...
XYL US Equity,Industrials
YUM US Equity,Consumer Discretionary
ZBH US Equity,Healthcare


In [22]:
smp500.to_csv("../sectors.csv")